<a href="https://colab.research.google.com/github/hmlewis-astro/street_network_deep_learning/blob/main/test_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Kaggle API credentials
#### **Note**: This is a one-time step and you don’t need to generate the credentials every time you download the dataset.
- Navigate to your Kaggle profile
- Click the "Account" tab
- Scroll down to the "API" section
- Click "Create New API Token"; a file named `kaggle.json` will be download which contains your username and API key

# Upload Kaggle API credentials to Google Colab
#### **Note**: Uploaded files will get deleted when this runtime is recycled.
- Upload the `kaggle.json` file that you just downloaded from Kaggle
- Run the following cell

In [11]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download balraj98/deepglobe-road-extraction-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
deepglobe-road-extraction-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
!unzip -q /content/deepglobe-road-extraction-dataset.zip -d /content/deep-globe

# Import packages and libraries

In [13]:
import os
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
class_dict_path = "/deep-globe/class_dict.csv"
class_dict = pd.read_csv(class_dict_path)
class_dict

,name,r,g,b
0,road,255,255,255
1,background,0,0,0


In [29]:
metadata_path = "/deep-globe/metadata.csv"
metadata = pd.read_csv(metadata_path)
metadata.head()

,image_id,split,sat_image_path,mask_path
0,100034,train,train/100034_sat.jpg,train/100034_mask.png
1,100081,train,train/100081_sat.jpg,train/100081_mask.png
2,100129,train,train/100129_sat.jpg,train/100129_mask.png
3,100703,train,train/100703_sat.jpg,train/100703_mask.png
4,100712,train,train/100712_sat.jpg,train/100712_mask.png


In [30]:
# get training/validation data, i.e., images with available road masks
metadata_train = metadata[metadata['split'] == 'train']
metadata_train.drop('split', inplace=True, axis=1)
metadata_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,image_id,sat_image_path,mask_path
0,100034,train/100034_sat.jpg,train/100034_mask.png
1,100081,train/100081_sat.jpg,train/100081_mask.png
2,100129,train/100129_sat.jpg,train/100129_mask.png
3,100703,train/100703_sat.jpg,train/100703_mask.png
4,100712,train/100712_sat.jpg,train/100712_mask.png


In [32]:
metadata_train.shape

(6226, 3)

In [31]:
# get test data, i.e., images without available road masks
#    combine the datasets defined as "validation" and "test", because the
#    "validation" set does not have road masks, so cannot actually be used
#    for validation
metadata_test = metadata[(metadata['split'] == 'valid') | 
                         (metadata['split'] == 'test')]
metadata_test.drop(['split', 'mask_path'], inplace=True, axis=1)
metadata_test.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,image_id,sat_image_path
6226,100794,valid/100794_sat.jpg
6227,100905,valid/100905_sat.jpg
6228,102867,valid/102867_sat.jpg
6229,10417,valid/10417_sat.jpg
6230,106553,valid/106553_sat.jpg


In [33]:
metadata_test.shape

(2344, 2)

In [34]:
data_path = "/deep-globe/"


In [37]:
metadata_train["sat_image_path"] = metadata_train["sat_image_path"] \
                                    .apply(lambda x: os.path.join(data_path,x))
metadata_train["mask_path"] = metadata_train["mask_path"] \
                                    .apply(lambda x: os.path.join(data_path,x))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
metadata_test["sat_image_path"] = metadata_test["sat_image_path"] \
                                    .apply(lambda x: os.path.join(data_path,x))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
